# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from botbidder import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('../UCBC 2024/Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../UCBC 2024/Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/contract-193200
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_

### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

from ddsolver import ddsolver
dds = ddsolver.DDSolver()

# you sit West and hold:
hand = 'J.J82.A98632.T52'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ["PAD_START","PAD_START","1H", "1N", "2S"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, seat=1, dealer=2, ddsolver=dds, verbose=False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

Loaded lib dds.dll
DDSolver being loaded - dds mode 1


[{'call': '3D',
  'insta_score': 0.307,
  'adjustment': 15,
  'expected_score': -6,
  'expected_tricks': 8.6},
 {'call': 'PASS',
  'insta_score': 0.631,
  'adjustment': 32,
  'expected_score': -88,
  'expected_tricks': 8.5}]

In [3]:
bid.samples

['Qxxxxxx.x.Qx.KJ8 J.J8x.A98xxx.Txx A9.AKTxxx.Tx.xxx KT8.Q9x.KJx.AQ9x 0.84082 \n 1H-1N-2S-P-P-P (9, 9.1)  \n 1H-1N-2S-3D-3H-P-P-P (8, 7.3) ',
 'KT8xxx.9.Qxx.K8x J.J8x.A98xxx.Txx Axx.Qxxxx.JT.Axx Q9x.AKTx.Kx.QJ9x 0.84082 \n 1H-1N-2S-P-P-P (9, 8.8)  \n 1H-1N-2S-3D-P-P-P (9, 8.1) ',
 'AT98xxx..KJx.xxx J.J8x.A98xxx.Txx Kx.KT9xxx.xx.KQ8 Qxx.AQxx.QT.AJ9x 0.83984 \n 1H-1N-2S-P-P-P (9, 9.1)  \n 1H-1N-2S-3D-P-P-3S-P-P-P (9, 9.1) ',
 'QT9xxxx.x.QJ.Axx J.J8x.A98xxx.Txx A8.AQ9xxx.Txx.9x Kxx.KTx.Kx.KQJ8x 0.81055 \n 1H-1N-2S-P-P-3C-P-P-P (8, 8.1)  \n 1H-1N-2S-3D-P-P-P (7, 8.3) ',
 'AQxxxx.x.xx.Q8xx J.J8x.A98xxx.Txx 9xx.AK9xx.KT.J9x KT8.QTxx.QJx.AKx 0.79883 \n 1H-1N-2S-P-P-P (9, 9.1)  \n 1H-1N-2S-3D-3S-P-P-P (9, 9.1) ',
 'AQT8xxx.xx.K.xxx J.J8x.A98xxx.Txx 9.AKT9x.QJTx.98x Kxxx.Qxx.xx.AKQJ 0.79492 \n 1H-1N-2S-P-P-P (8, 8.4)  \n 1H-1N-2S-3D-P-3N-P-P-P (6, 5.7) ',
 'QT9xxxx.Qx.x.Qxx J.J8x.A98xxx.Txx Kx.T9xxx.KQx.KJx A8x.AKx.JTx.A98x 0.78906 \n 1H-1N-2S-P-P-P (8, 8.1)  \n 1H-1N-2S-3D-P-P-P (9, 9.7) ',
 '

In [4]:
auction = ["PAD_START","PAD_START","1H", "1N", "2S","3D","3S","PASS","4S"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS', 'insta_score': 0.911}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [5]:
bid.samples

[]